#### 데이터
- https://www.kaggle.com/c/word2vec-nlp-tutorial/data?select=unlabeledTrainData.tsv.zip

- sentiment 0은 

In [2]:
import pandas as pd

# quoting(인용) : 3개의 쌍따옴표를 무시하도록 한다.
# delimiter(구분자) : \t는 필드가 탭으로 구분되는 것을 의미
# header : 0은 파일의 첫 번째 줄에 열 이름이 있음을 의미

train = pd.read_csv('word2vec/labeledTrainData.tsv', header=0, delimiter='\t',
                   quoting=3)

test = pd.read_csv('word2vec/testData.tsv', header=0, delimiter='\t',
                   quoting=3)

In [3]:
train.shape

(25000, 3)

In [4]:
test.shape

(25000, 2)

In [5]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [6]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [7]:
# train data에 어떤 컬럼이 있는지 확인

train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [8]:
test.columns.values

array(['id', 'review'], dtype=object)

In [9]:
# 결측치가 없는지 확인

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [10]:
train.describe()

,sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


In [12]:
# 
train['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [17]:
# 리뷰가 어떻게 작성되었는지 보기
# 리뷰를 보면 글만 있는것이 아니라 여러가지가 섞여 있기에 정제가 필요!
train['review'][0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

### 데이터 정제

1. BeautifulSoup을 통해 HTML태그 제거
2. 정규표현식으로 알파벳 이외의 문자를 공백으로 치환
3. NLLTK데이터를 사용해 불용어를 제거
4. 어간추출(스테밍 stemming)과 음소표기법(Lemmatizing)의 개념을 이해하고 SnowballStemmer를 통해 어간 추출

In [19]:
from bs4 import BeautifulSoup

# 
examplel = BeautifulSoup(train['review'][0], "html5lib") # 정제된 것 
print(train['review'][0][:700]) # 데이터 정제가 안 된것
examplel.get_text()[:700]

# 태그가 삭제된 것을 확인

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely lik


'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyw'

In [20]:
# 정규표현식을 사용하여 특수문자 제거

import re

letters_only = re.sub('[^a-zA-Z]', ' ', examplel.get_text())
# 소문자와 대문자가 아닌 것을 공백으로 대체
letters_only[:700]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyw'

In [21]:
# 모두 소문자로 변환
lower_case = letters_only.lower()

# 문자를 나눈다. => 토큰화
words = lower_case.split() # 공백으로 분리
print(len(words))
words[:10]

437


['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with']

###  불용어 제거(Stopword Removal)

In [24]:
import nltk
from nltk.corpus import stopwords
stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [26]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')
words = [stemmer.stem(w) for w in words]
words[:10]

# 스테밍 이란 runs, runining, run을 다 run으로 만드는 것이다.

['with', 'all', 'this', 'stuff', 'go', 'down', 'at', 'the', 'moment', 'with']

### Lemmatization 음소표기법
- 동음이의어를 문맥에 따라 다른 의미를 가지는 단어를 앞뒤 문맥을 보고 식별

In [35]:
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

print(wordnet_lemmatizer.lemmatize('fly'))
print(wordnet_lemmatizer.lemmatize('flies'))

words = [wordnet_lemmatizer.lemmatize(w) for w in words]
# 처리 후 단어
words[:10]

fly
fly


['with', 'all', 'this', 'stuff', 'go', 'down', 'at', 'the', 'moment', 'with']

---
- 여기까지가 html 태그 제거, 문자를 원형으로 변형하고 앞뒤 문맥에 따라 단어뜻을 구별

####  위에서 한 작업을 전체 데이터에 적용하기 위한 함수화 

In [38]:
def review_to_words(raw_review):
    # html 태그 제거
    review_text = BeautifulSoup(raw_review, "html.parser").get_text()
    # 영문자가 아닌 문자를 공백처리
    letters_only = re.sub('[^a-zA-Z]', " ", review_text)
    # 모든 문자 소문자로 변환
    words = letters_only.lower().split()
    # 파이썬에선 list보다 set로 찾는게 훨씬 빠르다.
    # stopwords를 set로 변환
    stops = set(stopwords.words('english'))
    # 불용어 제거
    meaningful_words = [w for w in words if not w in stops]
    # 어간 추출
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 공백으로 구분된 문자열로 결함하여 결과를 반환
    return (' '.join(stemming_words))

In [39]:
# 첫번째 리뷰만 보기
clean_review = review_to_words(train['review'][0]) 
clean_review

'stuff go moment mj start listen music watch odd documentari watch wiz watch moonwalk mayb want get certain insight guy thought realli cool eighti mayb make mind whether guilti innoc moonwalk part biographi part featur film rememb go see cinema origin releas subtl messag mj feel toward press also obvious messag drug bad kay visual impress cours michael jackson unless remot like mj anyway go hate find bore may call mj egotist consent make movi mj fan would say made fan true realli nice actual featur film bit final start minut exclud smooth crimin sequenc joe pesci convinc psychopath power drug lord want mj dead bad beyond mj overheard plan nah joe pesci charact rant want peopl know suppli drug etc dunno mayb hate mj music lot cool thing like mj turn car robot whole speed demon sequenc also director must patienc saint came film kiddi bad sequenc usual director hate work one kid let alon whole bunch perform complex danc scene bottom line movi peopl like mj one level anoth think peopl stay

In [40]:
# 전체 데이터 개수

tot_reviews = train['review'].size
tot_reviews

25000

In [ ]:

"""
clean_train_reviews = []
캐글 튜토리얼에는 range가 xrange로 되어있지만 
여기에서는 python3를 사용하기 때문에 range를 사용했다.
"""
# for i in range(0, num_reviews):
#     clean_train_reviews.append( review_to_words(train['review'][i]))
"""
하지만 위 코드는 어느 정도 실행이 되고 있는지 알 수가 없어서
5000개 단위로 상태를 찍도록 개선했다.
"""
# clean_train_reviews = []
# for i in range(0, num_reviews):
#     if (i + 1)%5000 == 0:
#         print('Review {} of {} '.format(i+1, num_reviews))
#     clean_train_reviews.append(review_to_words(train['review'][i]))
    
"""
그리고 코드를 좀 더 간결하게 하기 위해 for loop를 사용하는 
대신 apply를 사용하도록 개선
"""    
# %time train['review_clean'] = train['review'].apply(review_to_words)
"""
코드는 한 줄로 간결해 졌지만 여전히 오래 걸림
"""
# CPU times: user 1min 15s, sys: 2.3 s, total: 1min 18s
# Wall time: 1min 20s

In [41]:
# 참고 : https://gist.github.com/yong27/7869662
# http://www.racketracer.com/2016/07/06/pandas-in-parallel/
from multiprocessing import Pool
import numpy as np

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    # 키워드 항목 중 workers 파라메터를 꺼냄
    workers = kwargs.pop('workers')
    # 위에서 가져온 workers 수로 프로세스 풀을 정의
    pool = Pool(processes=workers)
    # 실행할 함수와 데이터프레임을 워커의 수 만큼 나눠 작업
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    # 작업 결과를 합쳐서 반환
    return pd.concat(list(result))

In [ ]:
# 멀티 프로세싱을 활용한 데이터 전처리

%time clean_train_reviews = apply_by_multiprocessing(\
    train['review'], review_to_words, workers=4)

### 워드 클라우드
- 단어의 빈도수 데이터를 가지고 있을 때 이용할 수 있는 시각하 방법

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

% matplotlib inline # 노트북 안에 그래프가 표시됨

def displaywordCloud(data=None, backgroundcolor='white',width=800,height=600):
                    wordcloud = WordCloud(stopwords=STOPWORDS,
                                         background_color = backgroundcolor,
                                         width=width, 
                                          height=height).generate(data)
                    pltfigure(figsize = (15,10))
                    plt.imshow(wordcloud)
                    plt.axis("off")
                    plt.show())


In [ ]:
# 모든 단어에 대한 워드 클라우드를 그려본다.
%time dispayWordCloud(" ".join(clean_train_reviews))

In [ ]:
%time dispayWordCloud(" ".join(clean_test_reviews))

In [ ]:
# 단어 수
train['num_words'] = clean_train_reviews.apply(lambda x: len(str(x).split()))
# 중복을 제거한 단어 수
train['num_uniq_words'] = clean_train_reviews.apply(lambda x: len(set(str(x).split())))

In [ ]:

import seaborn as sns

fig, axes = plt.subplots(ncols=2)
fig.set_size_inches(18, 6)
print('리뷰별 단어 평균 값 :', train['num_words'].mean())
print('리뷰별 단어 중간 값', train['num_words'].median())
sns.distplot(train['num_words'], bins=100, ax=axes[0])
axes[0].axvline(train['num_words'].median(), linestyle='dashed')
axes[0].set_title('리뷰별 단어 수 분포')

print('리뷰별 고유 단어 평균 값 :', train['num_uniq_words'].mean())
print('리뷰별 고유 단어 중간 값', train['num_uniq_words'].median())
sns.distplot(train['num_uniq_words'], bins=100, color='g', ax=axes[1])
axes[1].axvline(train['num_uniq_words'].median(), linestyle='dashed')
axes[1].set_title('리뷰별 고유한 단어 수 분포')